<a href="https://colab.research.google.com/github/kevin-rn/Grounding-LM/blob/main/fact_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Extract sentences from cnn dailymail articles and index them. Use claim detection or evidence sentence selection models to achieve this. For each summary generated from model consider it to be a claim and retrieve closed sentences from index. Use an out of box stance detection model to verify the summary against retrieved evidences.  


In [8]:
import os
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Grounding_LM/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/Grounding_LM/'
/content/drive/MyDrive/Grounding_LM


In [9]:
%pip install -q transformers
%pip install -q sentence-transformers
%pip install -q -U annoy

In [10]:
from annoy import AnnoyIndex
import ast
from collections import Counter
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, GPT2LMHeadModel, GPT2Tokenizer
import torch
from tqdm.auto import tqdm
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import random
import time

nltk.download('punkt')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tqdm.pandas()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Load data

In [ ]:
df_t5_cnn = pd.read_csv("results/generated summaries/t5_large_cnn_dailymail.csv", index_col=0)
df_t5_cnn.head()

,text,summary,id,generated
0,(CNN)The Palestinian Authority officially beca...,Membership gives the ICC jurisdiction over all...,f001ec5c4704938247d27a44948eebb37ae98d01,The Palestinians have become a member of the I...
1,(CNN)Never mind cats having nine lives. A stra...,"Theia, a bully breed mix, was apparently hit b...",230c522854991d053fe98a718b1defa077a8efef,A dog that was apparently buried alive after b...
2,"(CNN)If you've been following the news lately,...",Mohammad Javad Zarif has spent more time with ...,4495ba8f3a340d97a9df1476f8a35502bcce1f69,It's been a busy week for Iran.
3,(CNN)Five Americans who were monitored for thr...,17 Americans were exposed to the Ebola virus w...,a38e72fed88684ec8d60dd5856282e999dc8c0ca,Five Americans who were being treated for Ebol...
4,(CNN)A Duke student has admitted to hanging a ...,Student is no longer on Duke University campus...,c27cf1b136cc270023de959e7ab24638021bc43f,A student at Duke University has admitted hang...


In [ ]:
df_halueval = pd.read_csv('data/halueval/summarization_data.csv')
total_docs = 500
sampled_df = df_halueval.sample(n=total_docs, random_state=42)
sampled_df['index'] = sampled_df.index
sampled_df.head()

,document,right_summary,hallucinated_summary,index
6252,Driving around in their mother's consular BMW ...,"Marc Wabafiyebazu, 15, bragged to officials th...",Brothers Marc and Jean Wabafiyebazu were arres...,6252
4684,Lance Armstrong has said the World Anti-Doping...,WADA director general David Howman said he was...,Lance Armstrong has apologized to the World An...,4684
1731,Andy King thinks his 50th goal for Leicester C...,Andy King scored his 50th goal to earn Leicest...,Leicester City secured a crucial win against W...,1731
4742,West Ham have announced a new five-year multi-...,West Ham have signed a new kit deal with Umbro...,West Ham have announced a partnership with Umb...,4742
4521,"At half-time, everything pointed to another hu...",George Ford scythed through the Leinster defen...,Bath's George Ford scored a hat-trick of tries...,4521


In [ ]:
def tokenize_sentences(df_input):
  df_input['sentences'] = df_input['document'].apply(sent_tokenize)

In [ ]:
# tokenize_sentences(df_t5_cnn)
tokenize_sentences(sampled_df)


### Claim detection

1. Load pre-trained claim detection model (BERT pretrained on Claimbuster dataset)
2. Split each source document text into sentences using NLTK's `sent_tokenize`
3. Extract claimworthy sentences from this

In [ ]:
claim_tokenizer = AutoTokenizer.from_pretrained("Nithiwat/bert-base_claimbuster")
claim_model = AutoModelForSequenceClassification.from_pretrained("Nithiwat/bert-base_claimbuster").to(device)

In [ ]:
def extract_claimworthy(sentences):
    tokenized_inputs = claim_tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = claim_model(**tokenized_inputs).logits
        logits = logits.cpu()
    label_indices = torch.nonzero(logits.argmax(dim=1) == 0).squeeze().cpu()
    # Prevent looping over 0d-tensor error.
    if label_indices.dim() == 0:
        label_indices = label_indices.unsqueeze(0)

    claimworthy = [sentences[idx] for idx in label_indices]
    return claimworthy

In [ ]:
# df_test['claims'] = df_test['sentences'].progress_apply(extract_claims)
# df_test.to_csv('claims.csv', index=False)

In [ ]:
sentences = extract_claimworthy(df_test['sentences'][0])

print(f"evidence: {' '.join(sentences)} \nclaim: {df_test['generated'][0]}")

evidence: The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. These are substantive commitments, which cannot be taken lightly," she said. Prosecutor Fatou Bensouda said her office would "conduct its analysis in full independence and impartiality." 
claim: The Palestinians have become a member of the International Criminal Court (ICC).


In [ ]:
df_test['text'][0]

'(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony

### Construct Index

1. Load sentence-transformers model to create text embeddings for sentences & paragraphs
2. Calculate embeddings for each claimworthy sentence
3. Store embeddings using ANNOY library for index and retrieval.

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2') # 384 dimensional dense vector space

In [ ]:
def index_annoy(df_input, df_name, embedding_dim = 384, number_of_trees=100):
  for doc_id, row in df_input.iterrows():
    embeddings = [model.encode(txt) for txt in row['sentences']]
    ann = AnnoyIndex(embedding_dim, metric = "angular")
    for index, embed in enumerate(embeddings):
        ann.add_item(index, embed)
    ann.build(number_of_trees)
    ann.save(f"data/{df_name}/annoy/{doc_id}_{df_name}.annoy")

In [ ]:
index_annoy(sampled_df, 'halueval')
sampled_df.to_csv(f'data/halueval/sample_df_{total_docs}.csv', index=False)

# Inference
1. Retrieve top-k source document claimworthy sentence embeddings from ANNOY for a given claim (generated summary).
2. Calculate cosine similarity between the given claim and the retrieved sentences and keep the ones above certain cosine similarity.
3. Load pre-trained fact-checking model and infer whether evidence supports, refutes or is neutral for the given claim.

In [19]:
class KnnSearch:
    def __init__(self, emb_dim=384):
        self.annoy = AnnoyIndex(384, metric="angular")
        self.model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
        self.emb_dim = emb_dim

    def get_embeddings_for_data(self, data_ls):
        embeddings = self.model.encode(data_ls)
        return embeddings

    def standardize_normalize_cosine_similarities(self, cosine_similarities):
        cosine_sims_norm = (cosine_similarities - np.min(cosine_similarities)) / (np.max(cosine_similarities) - np.min(cosine_similarities))
        cosine_sims_norm = 0.5 + (cosine_sims_norm - np.mean(cosine_sims_norm)) / np.std(cosine_sims_norm)
        return cosine_sims_norm

    def max_normalize_cosine_similarities(self, cosine_similarities):
        return 1 / np.max(cosine_similarities) * cosine_similarities.squeeze(axis=1)

    def max_normalize_cosine_similarities_pairwise(self, cosine_similarities):
        cosine_sims_norm = np.copy(cosine_similarities)
        np.fill_diagonal(cosine_sims_norm, np.NaN)
        cosine_sims_norm = (cosine_similarities - np.nanmin(cosine_similarities, axis=0)) / (np.nanmax(cosine_similarities, axis=0) - np.nanmin(cosine_similarities, axis=0))
        cosine_sims_norm = 0.5 + (cosine_sims_norm - np.nanmean(cosine_sims_norm, axis=0)) / np.nanstd(cosine_sims_norm, axis=0)
        return cosine_sims_norm

    def get_top_nn_neighbours(self, df_name, df_input, df_index, claim, k, beta):
        annoy_index = df_input['index'][df_index]
        self.annoy.load(f"data/{df_name}/annoy/{annoy_index}_{df_name}.annoy")

        new_emb = self.model.encode(claim)
        top_matches = self.annoy.get_nns_by_vector(new_emb, k)
        evidence_sentences =  [df_input["sentences"][df_index][i] for i in top_matches]
        evidence_embeddings = self.get_embeddings_for_data(evidence_sentences)
        # top_sentences = []
        # for idx, similarity in sorted(enumerate(text_sims[0]), key=lambda x: x[1], reverse=True):
        #     if similarity > beta:
        #       top_sentences.append(evidence_sentences[idx])
        # return top_sentences

        text_sims = cosine_similarity(evidence_embeddings,[new_emb]).tolist()
        candidate_sims = cosine_similarity(evidence_embeddings)
        text_sims_norm = self.standardize_normalize_cosine_similarities(text_sims)
        phrase_sims_norm = self.max_normalize_cosine_similarities_pairwise(candidate_sims)

        selected_data_indices = []
        data_len = len(evidence_sentences)
        unselected_data_indices = list(range(data_len))

        best_idx = np.argmax(text_sims)
        selected_data_indices.append(best_idx)
        unselected_data_indices.remove(best_idx)

        # Select top N data
        for _ in range(min(data_len, k) - 1):
            unselected_data_distances_to_text = text_sims_norm[unselected_data_indices, :]
            unselected_data_distances_pairwise = phrase_sims_norm[unselected_data_indices][:,selected_data_indices]
            # if dimension of data distances is 1 we add additional axis to the end
            if unselected_data_distances_pairwise.ndim == 1:
                unselected_data_distances_pairwise = np.expand_dims(unselected_data_distances_pairwise, axis=1)

            # find new candidate with MMR retrieval
            idx = int(np.argmax(beta * unselected_data_distances_to_text - (1 - beta) * np.max(unselected_data_distances_pairwise, axis=1).reshape(-1, 1)))
            best_idx = unselected_data_indices[idx]

            # select new best phrase and update selected/unselected phrase indices list
            selected_data_indices.append(best_idx)
            unselected_data_indices.remove(best_idx)
            top_sent = [evidence_sentences[i] for i in selected_data_indices]

        return top_sent


In [20]:
knn = KnnSearch()
checkpoint = 'Dzeniks/roberta-fact-check'
factcheck_model = AutoModelForSequenceClassification.from_pretrained(checkpoint).to(device)
factcheck_tokenizer = AutoTokenizer.from_pretrained(checkpoint)
label_mapping = ['support', 'refute', 'neutral']

def fact_check_split_sent(claim, evidences):
    factcheck_model.eval()
    labels = []
    for evidence in evidences:
      features = factcheck_tokenizer.encode_plus(claim, evidence, truncation=True, return_tensors="pt", max_length=512).to(device)
      with torch.no_grad():
        prediction = factcheck_model(**features).logits
        logits = prediction.cpu().numpy()
        result = label_mapping[logits.argmax().item()]
        labels.append(result)
    return labels

def fact_check_split_claim(claims, evidences):
    factcheck_model.eval()
    results = []
    for claim in claims:
      labels = []
      for evidence in evidences:
        features = factcheck_tokenizer.encode_plus(claim, evidence, truncation=True, return_tensors="pt", max_length=512).to(device)
        with torch.no_grad():
          prediction = factcheck_model(**features).logits
          logits = prediction.cpu().numpy()
          result = label_mapping[logits.argmax().item()]
          labels.append(result)
      vote_counts = Counter(labels)
      majority_vote = vote_counts.most_common(1)[0][0]
      results.append(majority_vote)
    return results

def fact_check_join_sent(claim, evidences):
    factcheck_model.eval()
    features = factcheck_tokenizer.encode_plus(claim, ' '.join(evidences), truncation=True, return_tensors="pt", max_length=512).to(device)
    with torch.no_grad():
      prediction = factcheck_model(**features).logits
      logits = prediction.cpu().numpy()
    label = label_mapping[logits.argmax().item()]
    return label

def multi_fact_check(df_name, df_input, colname, k, beta, fact_type):
  stances, times, top_k = [], [], []
  for idx in df_input.index:
    start_time = time.time()
    claim = df_input[colname][idx]

    # Retrieve sentences
    top_sent = knn.get_top_nn_neighbours(df_name=df_name, df_input=df_input, df_index=idx, claim=claim, k=k, beta=beta)
    top_k.append(top_sent)

    # Use sentences as evidence and summary as claim for factchecking
    match fact_type:
      case 'split_sent':
        labels = fact_check_split_sent(claim, top_sent)
        # Majority vote on labels
        vote_counts = Counter(labels)
        majority_vote = vote_counts.most_common(1)[0][0]
        stances.append(majority_vote)

      case 'join_sent':
        label = fact_check_join_sent(claim, top_sent)
        stances.append(label)

      case other:
        labels = fact_check_split_claim(claims, top_sent)
        if 'refute' in labels:
          stances.append('refute')
        else:
          stances.append('support')

    elapsed_time = time.time() - start_time
    times.append(elapsed_time)
  return stances, times, top_k


In [ ]:
# Load sampled Halueval data
# sampled_df = pd.read_csv('data/sample_df.csv')
# sampled_df['sentences'] = sampled_df['sentences'].apply(ast.literal_eval)

# # Get closest evidence sentences and infer labels support or refute
# results = knn.get_top_nn_neighbours(df_name='halueval', df_input=sampled_df, df_index=2, claim=sampled_df['right_summary'][2], k=15, beta=0.7)
# labels = fact_check(sampled_df['right_summary'][2], results)
# # Majority vote
# vote_counts = Counter(labels)
# majority_vote = vote_counts.most_common(1)[0][0]
# print(f"Final label: {majority_vote}\n evidence: {results}\n claim: {sampled_df['right_summary'][2]}")


# label = fact_check_joint(sampled_df['right_summary'][2], results)
# print(f"Label: {label}\n evidence: {results}\n claim: {sampled_df['right_summary'][2]}")

Final label: refute
 evidence: ['Andy King thinks his 50th goal for Leicester City could prove to be his most important yet.', 'That proves to everyone - not just ourselves - that we mean business.’ Leicester manager Nigel Pearson praised King, who he brought off the bench with 12 minutes to play and said his team were finally getting the results they deserved after several games this season where they have drawn or lost despite dominating.', 'Andy King was the hero as Premier League strugglers Leicester City struck late to earn a vital three points.', "Pearson congratulates goalscorer Cambiasso after the final whistle of his side's win against the Hammers.", 'King is joined by David Nugent to celebrate his goal, the 50th he has scored for his club.', '‘I’ve scored some pretty crucial goals before, but we might look back at the end of the season and that might turn out to be the most important one yet,’ said King,.', "Coming in the 86th minute of Saturday's match it was by no means his

In [29]:
k_vals = [3, 5, 10, 15]
beta_vals = [0.7, 0.9, 1.0]
fact_types = ['split_sent', 'join_sent']
# fact_types = ['split_sent', 'join_sent', 'split_claim']
total_docs = 500

df_basis = pd.read_csv(f'data/halueval/sample_df_{total_docs}.csv')
df_basis['right_summary_sent'] = df_basis['right_summary'].apply(sent_tokenize)
df_basis['hallucinated_summary_sent'] = df_basis['hallucinated_summary'].apply(sent_tokenize)
df_basis['sentences'] = df_basis['sentences'].apply(ast.literal_eval)

for fact_type in tqdm(fact_types, desc="Main Loop"):
  for kv in tqdm(k_vals, desc="K-Values Loop", leave=False):
    for bv in tqdm(beta_vals, desc="Beta-Values Loop", leave=False):
      sampled_df = df_basis.copy(deep=True)

      stances, times, top_k = multi_fact_check('halueval', sampled_df, 'right_summary', kv, bv, fact_type)
      sampled_df['right_stance'] = stances
      sampled_df['right_inference_time'] = times
      sampled_df['right_top_k'] = top_k

      stances, times, top_k = multi_fact_check('halueval', sampled_df, 'hallucinated_summary', kv, bv, fact_type)
      sampled_df['hallucinated_stance'] = stances
      sampled_df['hallucinated_inference_time'] = times
      sampled_df['hallucinated_top_k'] = top_k

      sampled_df.drop(columns=['sentences'], inplace=True)
      sampled_df.to_csv(f'results/halueval/sampled_{total_docs}/sampled_k{kv}_b{bv}_{fact_type}.csv', index=False)

Main Loop:   0%|          | 0/1 [00:00<?, ?it/s]

K-Values Loop:   0%|          | 0/4 [00:00<?, ?it/s]

Beta-Values Loop:   0%|          | 0/3 [00:00<?, ?it/s]

Beta-Values Loop:   0%|          | 0/3 [00:00<?, ?it/s]

Beta-Values Loop:   0%|          | 0/3 [00:00<?, ?it/s]

Beta-Values Loop:   0%|          | 0/3 [00:00<?, ?it/s]

# Analysis stances

In [31]:
df_stances = pd.DataFrame(columns=['Name', 'FP', 'FN', 'Precision', 'Recall', 'F1'])
fact_types = ['split_sent', 'join_sent', 'split_claim']
for kv in k_vals:
  for bv in beta_vals:
    for fact_type in fact_types:
      df_temp = pd.read_csv(f'results/halueval/sampled_{total_docs}/sampled_k{kv}_b{bv}_{fact_type}.csv')
      fp = df_temp['right_stance'].value_counts().get('refute', 0)
      fn = df_temp['hallucinated_stance'].value_counts().get('support', 0)
      tp = total_docs - fp
      tn = total_docs - fn

      precision = tp / (tp + fp) if (tp + fp) > 0 else 0
      recall = tp / (tp + fn) if (tp + fn) > 0 else 0
      f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

      df_stances = pd.concat([df_stances, pd.DataFrame({'Name': [f'{fact_type}_k{kv}_b{bv}'], 'FP': [fp], 'FN': [fn], 'Precision': [precision], 'Recall': [recall], 'F1': [f1_score]})], ignore_index=True)

df_stances.to_csv(f'results/halueval/sampled_{total_docs}/stance_analysis.csv', index=False)
df_stances

,Name,FP,FN,Precision,Recall,F1
0,split_sent_k3_b0.7,165,365,0.670,0.478571,0.558333
1,join_sent_k3_b0.7,69,409,0.862,0.513095,0.643284
2,split_claim_k3_b0.7,463,97,0.074,0.276119,0.116719
3,split_sent_k3_b0.9,165,365,0.670,0.478571,0.558333
4,join_sent_k3_b0.9,69,409,0.862,0.513095,0.643284
5,split_claim_k3_b0.9,463,97,0.074,0.276119,0.116719
6,split_sent_k3_b1.0,165,365,0.670,0.478571,0.558333
7,join_sent_k3_b1.0,69,410,0.862,0.512485,0.642804
8,split_claim_k3_b1.0,463,97,0.074,0.276119,0.116719
9,split_sent_k5_b0.7,209,337,0.582,0.463376,0.515957
